In [1]:
import os
import sys

import numpy as np
import torch

sys.path.append(os.path.join(os.path.dirname("__file__"), ".."))

## Dataset

In [2]:
from modules.data import get_data

dataset = get_data("Cora") # no attack get_data("Cora", "n:10")
dataset

Cora()

## Training

In [3]:
from models.gat import GIBGAT
from models.gcn import GIBGCN

from modules.train import train_node_level

def print_results(result_dict):
    if "train" in result_dict:
        print("Train accuracy: %4.2f%%" % (100.0 * result_dict["train"]))
    if "val" in result_dict:
        print("Val accuracy:   %4.2f%%" % (100.0 * result_dict["val"]))
    print("Test accuracy:  %4.2f%%" % (100.0 * result_dict["test"]))
    
class Config(dict):
    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)
        self.__dict__ = self

### GCN

In [4]:
conf_GIBGCN = Config(
    exp_name="GGCN_CORA",
    model=GIBGCN(dataset.num_features, dataset.num_classes, latent_size=8),
    model_name="GIB-GCN",
    dataset_name="Cora",
    lr=0.03,
    weight_decay=5e-4,
    beta1=0.001,
    beta2=0.01,
    CHECKPOINT_PATH="../saved_models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    loss_type=dataset.loss,
)
conf_GIBGCN

{'exp_name': 'GGCN_CORA',
 'model': GIBGCN(
   (conv1): GCNConv(1433, 8)
   (conv2): GCNConv(8, 7)
 ),
 'model_name': 'GIB-GCN',
 'dataset_name': 'Cora',
 'lr': 0.03,
 'weight_decay': 0.0005,
 'beta1': 0.001,
 'beta2': 0.01,
 'CHECKPOINT_PATH': '../saved_models',
 'device': device(type='cuda'),
 'loss_type': 'softmax'}

In [5]:
# pl_GIBGCN, result_GIBGCN = train_node_level(
#     conf_GIBGCN,
#     dataset,
# )

# print_results(result_GIBGCN)

In [6]:
conf_GIBGAT = Config(
    exp_name="GIBGAT_CORA",
    model=GIBGAT(dataset.num_features, dataset.num_classes, latent_size=8),
    model_name="GIB-GAT",
    struct_dropout_mode=("standard", 0.6), # struct_dropout_mode = ("DNsampling",'multi-categorical-sum',1,3,2) GIB CAT
    dataset_name="Cora",
    lr=0.03,
    weight_decay=5e-4,
    beta1=0.001,
    beta2=0.01,
    CHECKPOINT_PATH="../saved_models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    loss_type=dataset.loss,
)
conf_GIBGAT

{'exp_name': 'GIBGAT_CORA',
 'model': GIBGAT(
   (conv1): GATConv(1433, 8, heads=1)
   (conv2): GATConv(8, 7, heads=1)
 ),
 'model_name': 'GIB-GAT',
 'struct_dropout_mode': ('standard', 0.6),
 'dataset_name': 'Cora',
 'lr': 0.03,
 'weight_decay': 0.0005,
 'beta1': 0.001,
 'beta2': 0.01,
 'CHECKPOINT_PATH': '../saved_models',
 'device': device(type='cuda'),
 'loss_type': 'softmax'}

In [7]:
# pl_GIBGAT, result_GIBGAT = train_node_level(
#     conf_GIBGAT,
#     dataset,
# )

# print_results(result_GIBGAT)

In [8]:
conf_GIBGAT = Config(
    exp_name="GIBGAT_CORA-DNsampling",
    model=GIBGAT(dataset.num_features, dataset.num_classes, latent_size=8, struct_dropout_mode = ("DNsampling",'multi-categorical-sum',1,3,2)),
    model_name="GIB-GAT-DNsampling",
    struct_dropout_mode = ("DNsampling",'multi-categorical-sum',1,3,2),
    dataset_name="Cora",
    lr=0.03,
    weight_decay=5e-4,
    beta1=0.001,
    beta2=0.01,
    CHECKPOINT_PATH="../saved_models",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    loss_type=dataset.loss,
)
conf_GIBGAT

{'exp_name': 'GIBGAT_CORA-DNsampling',
 'model': GIBGAT(
   (conv1): GATConv(1433, 8, heads=1)
   (conv2): GATConv(8, 7, heads=1)
 ),
 'model_name': 'GIB-GAT-DNsampling',
 'struct_dropout_mode': ('DNsampling', 'multi-categorical-sum', 1, 3, 2),
 'dataset_name': 'Cora',
 'lr': 0.03,
 'weight_decay': 0.0005,
 'beta1': 0.001,
 'beta2': 0.01,
 'CHECKPOINT_PATH': '../saved_models',
 'device': device(type='cuda'),
 'loss_type': 'softmax'}

In [9]:
pl_GIBGAT, result_GIBGAT = train_node_level(
    conf_GIBGAT,
    dataset,
)

print_results(result_GIBGAT)

Global seed set to 42
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


/home/step/.miniconda3/envs/gib/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | GIBGAT | 11.6 K
---------------------------------
11.6 K    Trainable params
0         Non-trainable params
11.6 K    Total params
0.046     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

TypeError: unsupported operand type(s) for +: 'NoneType' and 'float'